In [31]:
import glam
import pandas as pd
import numpy as np
import os.path

import matplotlib.pyplot as plt

In [32]:
import pymc3 as pm

In [33]:
np.random.seed(23) # from random.org

# 3.1. Hierarchical GLAM estimation and out of sample prediction

## Load data

In [34]:
# Load data
sufix = 'hierarchical_More_NoBin_Gamma-11_NUTS_33'
#data = pd.read_csv('data/PF2019_data/GlamDataPF2019_More_NoBin_33.csv')
data = pd.read_csv('data/PF2019_data/GlamDataFF2018_Like_NoBin_TEST.csv')

# Subset only necessary columns
data = data[['subject', 'trial', 'choice', 'rt',
         'item_value_0', 'item_value_1',
         'gaze_0', 'gaze_1']]
data.head()

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,0.0,0.0,0.0,2009.0,110.0,131.0,0.568396,0.431604
1,0.0,1.0,0.0,3371.0,47.0,50.0,0.762332,0.237668
2,0.0,2.0,1.0,1700.0,50.0,44.0,0.446809,0.553191
3,0.0,3.0,1.0,7466.0,57.0,50.0,0.532352,0.467648
4,0.0,4.0,1.0,1889.0,42.0,50.0,0.529736,0.470264


## Split data in training and test sets

In [35]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for subject in data.subject.unique():
    subject_data = data[data['subject'] == subject].copy().reset_index(drop=True)
    n_trials = len(subject_data)
    
    subject_train = subject_data.iloc[np.arange(0, n_trials, 2)].copy()
    subject_test = subject_data.iloc[np.arange(1, n_trials, 2)].copy()

    test_data = pd.concat([test_data, subject_test])
    train_data = pd.concat([train_data, subject_train])

test_data.to_csv(str('data/PF2019_data/GlamDataPF2019_preprocessed_test'+sufix+'.csv'))
train_data.to_csv(str('data/PF2019_data/GlamDataPF2019_preprocessed_train'+sufix+'.csv'))

print('Split data into training ({} trials) and test ({} trials) sets...'.format(len(train_data), len(test_data)))

Split data into training (1920 trials) and test (1920 trials) sets...


In [36]:
#train_data1 = train_data.loc[(train_data['subject'] == 15) | (train_data['subject'] == 15)  ] 
train_data1.rt.round(1)

0       5398.0
2      10254.0
4       3581.0
6       2128.0
8       2434.0
10      3431.0
12      2264.0
14      4420.0
16      2578.0
18      4365.0
20      5608.0
22      4357.0
24      3293.0
26      2743.0
28      2364.0
30      4363.0
32      5752.0
34      2780.0
36      6016.0
38     13395.0
40      2537.0
42      1619.0
44      1357.0
46      1295.0
48      1539.0
50      1641.0
52      1399.0
54      2077.0
56      2894.0
58      2923.0
60      1653.0
62      1651.0
64      1974.0
66      2072.0
68      2096.0
70      1402.0
72      1342.0
74      1656.0
76      1997.0
78      1851.0
80      5859.0
82      2559.0
84      2053.0
86      1325.0
88      2057.0
90      1676.0
92      1523.0
94      3258.0
96      1785.0
98      1620.0
100     2821.0
102     2143.0
104     1635.0
106     1497.0
108     1409.0
110     2282.0
112     1484.0
114     1386.0
116     1569.0
118     2294.0
Name: rt, dtype: float64

## Hierarchical GLAM estimation

In [37]:
train_data1

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,15.0,0.0,0.0,5398.0,110.0,131.0,0.767765,0.232235
2,15.0,2.0,1.0,10254.0,50.0,44.0,0.780964,0.219036
4,15.0,4.0,1.0,3581.0,42.0,50.0,0.347021,0.652979
6,15.0,6.0,0.0,2128.0,78.0,80.0,0.573778,0.426222
8,15.0,8.0,0.0,2434.0,50.0,48.0,0.682827,0.317173
10,15.0,10.0,0.0,3431.0,46.0,50.0,0.465171,0.534829
12,15.0,12.0,0.0,2264.0,50.0,56.0,0.508392,0.491608
14,15.0,14.0,1.0,4420.0,110.0,124.0,0.188235,0.811765
16,15.0,16.0,1.0,2578.0,64.0,80.0,0.235066,0.764934
18,15.0,18.0,1.0,4365.0,101.0,110.0,0.473769,0.526231


### 1. full GLAM

In [38]:
# Fitting full GLAM
print('Fitting full GLAM hierarchically...')

glam_full = glam.GLAM(train_data1)

if not os.path.exists(str('results/estimates/glam_PF2019_full_hierarchical_cv'+sufix+'.npy')):
    glam_full.make_model('hierarchical', gamma_bounds=(-1, 1), t0_val=0)
    glam_full.fit(method='ADVI', n_advi=200000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_full.estimates = np.load(str('results/estimates/glam_PF2019_full_hierarchical_cv'+sufix+'.npy'))   

Fitting full GLAM hierarchically...
Generating hierarchical model for 1 subjects...


/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


IndexError: index 15 is out of bounds for axis 0 with size 1

In [ ]:
# Save parameter estimates
np.save(str('results/estimates/glam_PF2019_nobias_hierarchical_cv'+sufix+'.npy'), glam_full.estimates)
pd.DataFrame(glam_full.estimates)

In [ ]:
# Compute WAICs
print('Computing WAIC scores for full model...')
if not os.path.exists(str('results/waic/glam_PF2019_full'+ sufix +'.npy')):
    # Note: DIC computation does not work for ADVI fitted models
    # But we are using WAIC
    glam_full.compute_waic()
else:
    print('  Found old DIC scores in "results/waic". Skipping WAIC computation...')
    glam_full.waic = np.load(str('results/waic/glam_PF2019_full'+ sufix +'.npy'))

# Compute WAICs
np.save(str('results/waic/glam_PF2019_full'+ sufix +'.npy'), glam_full.waic)

In [ ]:
glam_full.waic

In [ ]:
# Compute LOO

glam_full.loo = pm.loo(trace=glam_full.trace, model=glam_full.model)
glam_full.loo
np.save(str('results/loo/glam_PF2019_full'+ sufix +'.npy'), glam_full.loo)

In [ ]:
glam_full.loo

In [ ]:
# Predictions
print('Predicting test set data using full GLAM...')
glam_full.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_PF2019_full_hierarchical_cv'+sufix+'.csv')):
    glam_full.predict(n_repeats=50)
    glam_full.prediction.to_csv(str('results/predictions/glam_PF2019_full_hierarchical_cv'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical full GLAM predictions in "results/predictions". Skipping prediction...')
    glam_full.prediction = pd.read_csv(str('results/predictions/glam_PF2019_full_hierarchical_cv'+sufix+'.csv'))

glam_full.prediction.head()

### 1. no-bias GLAM

In [ ]:
# Fitting no-bias GLAM
print('Fitting no-bias GLAM hierarchically...')

glam_nobias = glam.GLAM(train_data)

if not os.path.exists(str('results/estimates/glam_PF2019_nobias_hierarchical_cv'+sufix+'.npy')):
    glam_nobias.make_model('hierarchical', gamma_val=1.0, t0_val=0)
    glam_nobias.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_nobias.estimates = np.load(str('results/estimates/glam_PF2019_nobias_hierarchical_cv'+sufix+'.npy'))
 

In [ ]:
   
# Save parameter estimates
np.save(str('results/estimates/glam_PF2019_nobias_hierarchical_cv'+sufix+'.npy'), glam_nobias.estimates)
pd.DataFrame(glam_nobias.estimates)

In [ ]:
# In case it is already fitted
params_part_like = pd.DataFrame.from_dict(glam_nobias.estimates.item(0))
params_part_like

In [ ]:
# Compute LOO

glam_nobias.loo = pm.loo(trace=glam_nobias.trace, model=glam_nobias.model)
glam_nobias.loo

np.save(str('results/loo/glam_PF2019_nobias'+ sufix +'.npy'), glam_nobias.loo
)

In [ ]:
# Predictions
print('Predicting test set data using no-bias GLAM...')
glam_nobias.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_PF2019_nobias_hierarchical_cv'+sufix+'.csv')):
    glam_nobias.predict(n_repeats=50)
    glam_nobias.prediction.to_csv(str('results/predictions/glam_PF2019_nobias_hierarchical_cv'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical no-bias GLAM predictions in "results/predictions". Skipping prediction...')
    glam_nobias.prediction = pd.read_csv(str('results/predictions/glam_PF2019_nobias_hierarchical_cv'+sufix+'.csv'))

glam_nobias.prediction.head()

## 2. Plot fit

In [ ]:
print('Close Figure to continue...')
glam.plot_fit(test_data, [glam_full.prediction]);
#glam.plot_fit(test_data, [glam_full.prediction,glam_nobias.prediction]);

plt.show()

## Parameters for full hierarchical model

In [ ]:
params_participant = glam_full.estimates
params_participant

In [ ]:
params_participant = pd.DataFrame.from_dict(glam_full.estimates.item(0))

In [ ]:
params_participant

In [ ]:
print ("Mean gamma " +  str(params_participant['gamma'].mean()))

In [ ]:
hist = params_participant[['SNR','gamma','tau','v']].hist(figsize = [20,3] , layout=[1,4],bins = 20)

## [END] 